# A self-Attentive model for Knowledge Tracing


## update:

### 2020-12-11: auc -> 0.765

- add state for test dataset

thanks for HDKIM wordk: https://www.kaggle.com/leadbest/sakt-with-state-updates

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import gc
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

## Load data

In [3]:
%%time
dtype = {'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16','content_type_id': 'int8','answered_correctly':'int8'}

train_df = pd.read_csv('train.csv', usecols=[1, 2, 3,4,7], dtype=dtype)
train_df.head()

Wall time: 36.1 s


,timestamp,user_id,content_id,content_type_id,answered_correctly
0,0,115,5692,0,1
1,56943,115,5716,0,1
2,118363,115,128,0,1
3,131167,115,7860,0,1
4,137965,115,7922,0,1


In [4]:
train_df = train_df[train_df.content_type_id == False]

#arrange by timestamp
train_df = train_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

## Preprocess

In [9]:
skills = train_df["content_id"].unique()
n_skill = len(skills)
print("number skills", len(skills))

number skills 13523


In [10]:
#直接生成了操作和对应的序列
group = train_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))

del train_df

In [9]:
group

user_id
115           ([5692, 5716, 128, 7860, 7922, 156, 51, 50, 78...
124           ([7900, 7876, 175, 1278, 2065, 2063, 2064, 336...
2746          ([5273, 758, 5976, 236, 404, 382, 405, 873, 53...
5382          ([5000, 3944, 217, 5844, 5965, 4990, 5235, 605...
8623          ([3915, 4750, 6456, 3968, 6104, 5738, 6435, 54...
                                    ...                        
2147470770    ([7900, 7876, 175, 1278, 2064, 2063, 2065, 336...
2147470777    ([7900, 7876, 175, 1278, 2065, 2063, 2064, 336...
2147481750    ([4137, 1270, 9261, 8201, 367, 378, 214, 6071,...
2147482216    ([3748, 4765, 5474, 9261, 4665, 5987, 6666, 56...
2147482888    ([6147, 4792, 5738, 6102, 4748, 7956, 6435, 92...
Length: 393656, dtype: object

In [11]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill, max_seq=100):
        super(SAKTDataset, self).__init__()
        #最大问题长度100以内
        self.max_seq = max_seq
        #question个数
        self.n_skill = n_skill
        self.samples = group
        
#         self.user_ids = [x for x in group.index]
        self.user_ids = []
        #把之前问题小于十个的用户剔除
        for user_id in group.index:
            q, qa = group[user_id]
            if len(q) < 10:
                #如果没有十个就结束当前循环，也就是不放入列表
                continue
            self.user_ids.append(user_id)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        #在序列模型里很常见的一个情况就是要对数据长度进行处理
        user_id = self.user_ids[index]
        q_, qa_ = self.samples[user_id]
        seq_len = len(q_)
        #这里按照最大长度（100）来设置0
        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        if seq_len >= self.max_seq:
            #大于最大长度，按最大长度进行截断
            q[:] = q_[-self.max_seq:]
            qa[:] = qa_[-self.max_seq:]
        else:
            #小于最大长度把序列长度以前的位置设为0
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
        #往右边移动一个单位，因为目前的question和answer都不能包含当前T时刻的结构，所以移动为t
        target_id = q[1:]
        label = qa[1:]
        #x仍然是历史的答题情况，用历史去预测未来 ，所以是1到 t-1
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill

        return x, target_id, label

In [17]:
#在前面补0，并且可以看出x比label和targetid要少一位，也就是即将要预测的那一位

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0, 21423,  7876,   175,  1278, 15587,  2065,  2063,
        3364,  3365, 16886,  2948,  2946,  2947, 16118,  2593, 16117,
       18015,  4120, 18219, 19639,  6173, 19893,  6878,  6877,  6880,
        6879,  7216,  7217,  7218,  7219, 18191,  4253, 19747,  9050])

In [18]:
len(train_dataset[0][0])

99

In [13]:
train, val = train_test_split(group, test_size=0.2)

train_dataset = SAKTDataset(train, n_skill)
train_dataloader = DataLoader(train_dataset, batch_size=2048, shuffle=True, num_workers=8)
del train

val_dataset = SAKTDataset(val, n_skill)
val_dataloader = DataLoader(val_dataset, batch_size=2048, shuffle=True, num_workers=8)
del val
# print(item[0])
# print(item[1])
# print(item[2])

## Define model

In [34]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=100, embed_dim=128):
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        #位置编码的词袋大小就是序列个数99
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        
        x = self.embedding(x)
        #这个是0-99的顺序，因为一共99维的序列，按照顺序进行编码
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        pos_x = self.pos_embedding(pos_id)
        #把原始编码和顺序编码合并
        x = x + pos_x
        #对问题id进行编码
        e = self.e_embedding(question_ids)
        #换位置以便做muti_att
        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        #这是用个上三角来挡住，避免出现用x2,x3,x4来预测x1的情况出现，即X2只能用X1，X3只能用X2,X1
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        #恢复准备入全接
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]
        #在这个地方可以把其他统计量拼接，再放入全连接层
        x = self.ffn(att_output)
        #残差结构
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


model = SAKTModel(n_skill, embed_dim=128)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

In [37]:
def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(train_iterator)
    for item in tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()
        target_mask = (target_id != 0)

        optim.zero_grad()
        output, atten_weight = model(x, target_id)
        
        output = torch.masked_select(output, target_mask)
        label = torch.masked_select(label, target_mask)
        
        loss = criterion(output, label)
        loss.backward()
        optim.step()
        train_loss.append(loss.item())
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.average(train_loss)

    return loss, acc, auc

In [38]:
def val_epoch(model, val_iterator, criterion, device="cpu"):
    model.eval()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(val_iterator)
    for item in tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()
        target_mask = (target_id != 0)

        with torch.no_grad():
            output, atten_weight = model(x, target_id)
        
        output = torch.masked_select(output, target_mask)
        label = torch.masked_select(label, target_mask)

        loss = criterion(output, label)
        train_loss.append(loss.item())

        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.average(train_loss)

    return loss, acc, auc

In [42]:
train_dataset[0][0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
       13651, 21383, 21445,   156, 13574, 13573, 21419, 21386, 13675,
         104, 13631, 21423, 21424,  7971, 13548, 13706, 21449,  7927,
       13527, 21507, 13568, 13708, 13578,  7876,     6,   172,  7898,
         175, 13623,  7859, 21677, 21678, 21679,  8157, 21681, 20890,
       20891, 20893, 20889,  7369, 20708,  7184,  7186,  1232,   357,
        1278,  2063, 15587, 15588,  3363,  3364,  3365,  2948,  2946])

In [13]:
epochs = 20

over_fit = 0
last_auc = 0
for epoch in range(epochs):
    train_loss, train_acc, train_auc = train_epoch(model, train_dataloader, optimizer, criterion, device)
    print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, train_loss, train_acc, train_auc))
    
    val_loss, avl_acc, val_auc = val_epoch(model, val_dataloader, criterion, device)
    print("epoch - {} val_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, val_loss, avl_acc, val_auc))
    
    if val_auc > last_auc:
        last_auc = val_auc
        over_fit = 0
    else:
        over_fit += 1
        
    
    if over_fit >= 2:
        print("early stop epoch ", epoch)
        break

  0%|          | 0/39 [00:00<?, ?it/s]

epoch - 0 train_loss - 0.63 acc - 0.652 auc - 0.671


  0%|          | 0/153 [00:00<?, ?it/s]

epoch - 0 val_loss - 0.59 acc - 0.684 auc - 0.729


  0%|          | 0/39 [00:00<?, ?it/s]

...

epoch - 13 val_loss - 0.57 acc - 0.701 auc - 0.759


  0%|          | 0/39 [00:00<?, ?it/s]

epoch - 14 train_loss - 0.57 acc - 0.705 auc - 0.764


loss - 0.5664: 100%|██████████| 39/39 [00:06<00:00,  5.78it/s]


epoch - 14 val_loss - 0.57 acc - 0.701 auc - 0.759
early stop epoch  14


## Test

In [14]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, skills, max_seq=100):
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.skills = skills
        self.n_skill = len(skills)
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)

        if user_id in self.samples.index:
            q_, qa_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_          
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[1:].copy()
        x += (qa[1:] == 1) * self.n_skill
        
        questions = np.append(q[2:], [target_id])
        
        return x, questions

In [15]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

In [16]:
import psutil

model.eval()

prev_test_df = None
MAX_SEQ = 100

for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM
    if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):
        print(psutil.virtual_memory().percent)
        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prev_test_df = prev_test_df[prev_test_df.content_type_id == False]
        prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))
        for prev_user_id in prev_group.index:
            prev_group_content = prev_group[prev_user_id][0]
            prev_group_ac = prev_group[prev_user_id][1]
            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_content), 
                                       np.append(group[prev_user_id][1],prev_group_ac))
 
            else:
                group[prev_user_id] = (prev_group_content,prev_group_ac)
            if len(group[prev_user_id][0])>MAX_SEQ:
                new_group_content = group[prev_user_id][0][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][1][-MAX_SEQ:]
                group[prev_user_id] = (new_group_content,new_group_ac)

    prev_test_df = test_df.copy()
    
    test_df = test_df[test_df.content_type_id == False]
    
    test_dataset = TestDataset(group, test_df, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs = []

    for item in tqdm(test_dataloader):
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()

        with torch.no_grad():
            output, att_weight = model(x, target_id)
        
        
        output = torch.sigmoid(output)
        output = output[:, -1]

        # pred = (output >= 0.5).long()
        # loss = criterion(output, label)

        # val_loss.append(loss.item())
        # num_corrects += (pred == label).sum().item()
        # num_total += len(label)

        # labels.extend(label.squeeze(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())
        
    test_df['answered_correctly'] =  outs
    
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 36.73it/s]

100%|██████████| 1/1 [00:00<00:00, 88.89it/s]
2it [00:00, 12.19it/s]
100%|██████████| 1/1 [00:00<00:00, 83.95it/s]

100%|██████████| 1/1 [00:00<00:00, 61.12it/s]


28.5
28.6
28.6


4it [00:00,  5.95it/s]
